In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/forecasting-sticker-sales/sample_submission.csv
/kaggle/input/forecasting-sticker-sales/train.csv
/kaggle/input/forecasting-sticker-sales/test.csv


# 01 / 20 / 2025
# TODO: Cross validation and using Mean Absolute Percentage Error

In [2]:
sticker_data = pd.read_csv('/kaggle/input/forecasting-sticker-sales/train.csv')
sticker_data.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,id,date,country,store,product,num_sold
0,0,2010-01-01,Canada,Discount Stickers,Holographic Goose,NaN
1,1,2010-01-01,Canada,Discount Stickers,Kaggle,973.0
2,2,2010-01-01,Canada,Discount Stickers,Kaggle Tiers,906.0
3,3,2010-01-01,Canada,Discount Stickers,Kerneler,423.0
4,4,2010-01-01,Canada,Discount Stickers,Kerneler Dark Mode,491.0


In [3]:
sticker_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230130 entries, 0 to 230129
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   id        230130 non-null  int64  
 1   date      230130 non-null  object 
 2   country   230130 non-null  object 
 3   store     230130 non-null  object 
 4   product   230130 non-null  object 
 5   num_sold  221259 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 10.5+ MB


# Because we are trying to predict the number sold, we cannot impute that data as it may interfere with our final results, so we'll drop all rows that have no num_sold data

In [4]:
sticker_data_cleaned = sticker_data[sticker_data['num_sold'].notna()]
sticker_data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 221259 entries, 1 to 230129
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   id        221259 non-null  int64  
 1   date      221259 non-null  object 
 2   country   221259 non-null  object 
 3   store     221259 non-null  object 
 4   product   221259 non-null  object 
 5   num_sold  221259 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 11.8+ MB


In [5]:
sticker_data['country'].unique()

array(['Canada', 'Finland', 'Italy', 'Kenya', 'Norway', 'Singapore'],
      dtype=object)

In [6]:
sticker_data['store'].unique()

array(['Discount Stickers', 'Stickers for Less', 'Premium Sticker Mart'],
      dtype=object)

In [7]:
sticker_data['product'].unique()

array(['Holographic Goose', 'Kaggle', 'Kaggle Tiers', 'Kerneler',
       'Kerneler Dark Mode'], dtype=object)

In [8]:
# Checking to see if there's any order among the different cards sold
sticker_data['product'].value_counts()

product
Holographic Goose     46026
Kaggle                46026
Kaggle Tiers          46026
Kerneler              46026
Kerneler Dark Mode    46026
Name: count, dtype: int64

Except for date, all features should be one-hot encoded and because we are predicting a continous variable with a large dataset ( > 100k), we will use the SGDRegressor.
We will drop id and date.

In [9]:
# See if our join was correct
df = sticker_data_cleaned.join(pd.get_dummies(sticker_data_cleaned['country'])).drop(columns='country')
df

,id,date,store,product,num_sold,Canada,Finland,Italy,Kenya,Norway,Singapore
1,1,2010-01-01,Discount Stickers,Kaggle,973.0,True,False,False,False,False,False
2,2,2010-01-01,Discount Stickers,Kaggle Tiers,906.0,True,False,False,False,False,False
3,3,2010-01-01,Discount Stickers,Kerneler,423.0,True,False,False,False,False,False
4,4,2010-01-01,Discount Stickers,Kerneler Dark Mode,491.0,True,False,False,False,False,False
5,5,2010-01-01,Stickers for Less,Holographic Goose,300.0,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
230125,230125,2016-12-31,Premium Sticker Mart,Holographic Goose,466.0,False,False,False,False,False,True
230126,230126,2016-12-31,Premium Sticker Mart,Kaggle,2907.0,False,False,False,False,False,True
230127,230127,2016-12-31,Premium Sticker Mart,Kaggle Tiers,2299.0,False,False,False,False,False,True
230128,230128,2016-12-31,Premium Sticker Mart,Kerneler,1242.0,False,False,False,False,False,True


In [10]:
sticker_data_cleaned = df
sticker_data_cleaned

,id,date,store,product,num_sold,Canada,Finland,Italy,Kenya,Norway,Singapore
1,1,2010-01-01,Discount Stickers,Kaggle,973.0,True,False,False,False,False,False
2,2,2010-01-01,Discount Stickers,Kaggle Tiers,906.0,True,False,False,False,False,False
3,3,2010-01-01,Discount Stickers,Kerneler,423.0,True,False,False,False,False,False
4,4,2010-01-01,Discount Stickers,Kerneler Dark Mode,491.0,True,False,False,False,False,False
5,5,2010-01-01,Stickers for Less,Holographic Goose,300.0,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
230125,230125,2016-12-31,Premium Sticker Mart,Holographic Goose,466.0,False,False,False,False,False,True
230126,230126,2016-12-31,Premium Sticker Mart,Kaggle,2907.0,False,False,False,False,False,True
230127,230127,2016-12-31,Premium Sticker Mart,Kaggle Tiers,2299.0,False,False,False,False,False,True
230128,230128,2016-12-31,Premium Sticker Mart,Kerneler,1242.0,False,False,False,False,False,True


In [11]:
df = sticker_data_cleaned.join(pd.get_dummies(sticker_data_cleaned['store'])).drop(columns='store')
df

,id,date,product,num_sold,Canada,Finland,Italy,Kenya,Norway,Singapore,Discount Stickers,Premium Sticker Mart,Stickers for Less
1,1,2010-01-01,Kaggle,973.0,True,False,False,False,False,False,True,False,False
2,2,2010-01-01,Kaggle Tiers,906.0,True,False,False,False,False,False,True,False,False
3,3,2010-01-01,Kerneler,423.0,True,False,False,False,False,False,True,False,False
4,4,2010-01-01,Kerneler Dark Mode,491.0,True,False,False,False,False,False,True,False,False
5,5,2010-01-01,Holographic Goose,300.0,True,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
230125,230125,2016-12-31,Holographic Goose,466.0,False,False,False,False,False,True,False,True,False
230126,230126,2016-12-31,Kaggle,2907.0,False,False,False,False,False,True,False,True,False
230127,230127,2016-12-31,Kaggle Tiers,2299.0,False,False,False,False,False,True,False,True,False
230128,230128,2016-12-31,Kerneler,1242.0,False,False,False,False,False,True,False,True,False


In [12]:
sticker_data_cleaned = df
sticker_data_cleaned

,id,date,product,num_sold,Canada,Finland,Italy,Kenya,Norway,Singapore,Discount Stickers,Premium Sticker Mart,Stickers for Less
1,1,2010-01-01,Kaggle,973.0,True,False,False,False,False,False,True,False,False
2,2,2010-01-01,Kaggle Tiers,906.0,True,False,False,False,False,False,True,False,False
3,3,2010-01-01,Kerneler,423.0,True,False,False,False,False,False,True,False,False
4,4,2010-01-01,Kerneler Dark Mode,491.0,True,False,False,False,False,False,True,False,False
5,5,2010-01-01,Holographic Goose,300.0,True,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
230125,230125,2016-12-31,Holographic Goose,466.0,False,False,False,False,False,True,False,True,False
230126,230126,2016-12-31,Kaggle,2907.0,False,False,False,False,False,True,False,True,False
230127,230127,2016-12-31,Kaggle Tiers,2299.0,False,False,False,False,False,True,False,True,False
230128,230128,2016-12-31,Kerneler,1242.0,False,False,False,False,False,True,False,True,False


In [13]:
df = sticker_data_cleaned.join(pd.get_dummies(sticker_data_cleaned['product'])).drop(columns='product')
df

,id,date,num_sold,Canada,Finland,Italy,Kenya,Norway,Singapore,Discount Stickers,Premium Sticker Mart,Stickers for Less,Holographic Goose,Kaggle,Kaggle Tiers,Kerneler,Kerneler Dark Mode
1,1,2010-01-01,973.0,True,False,False,False,False,False,True,False,False,False,True,False,False,False
2,2,2010-01-01,906.0,True,False,False,False,False,False,True,False,False,False,False,True,False,False
3,3,2010-01-01,423.0,True,False,False,False,False,False,True,False,False,False,False,False,True,False
4,4,2010-01-01,491.0,True,False,False,False,False,False,True,False,False,False,False,False,False,True
5,5,2010-01-01,300.0,True,False,False,False,False,False,False,False,True,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230125,230125,2016-12-31,466.0,False,False,False,False,False,True,False,True,False,True,False,False,False,False
230126,230126,2016-12-31,2907.0,False,False,False,False,False,True,False,True,False,False,True,False,False,False
230127,230127,2016-12-31,2299.0,False,False,False,False,False,True,False,True,False,False,False,True,False,False
230128,230128,2016-12-31,1242.0,False,False,False,False,False,True,False,True,False,False,False,False,True,False


In [14]:
sticker_data_cleaned = df
sticker_data_cleaned

,id,date,num_sold,Canada,Finland,Italy,Kenya,Norway,Singapore,Discount Stickers,Premium Sticker Mart,Stickers for Less,Holographic Goose,Kaggle,Kaggle Tiers,Kerneler,Kerneler Dark Mode
1,1,2010-01-01,973.0,True,False,False,False,False,False,True,False,False,False,True,False,False,False
2,2,2010-01-01,906.0,True,False,False,False,False,False,True,False,False,False,False,True,False,False
3,3,2010-01-01,423.0,True,False,False,False,False,False,True,False,False,False,False,False,True,False
4,4,2010-01-01,491.0,True,False,False,False,False,False,True,False,False,False,False,False,False,True
5,5,2010-01-01,300.0,True,False,False,False,False,False,False,False,True,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230125,230125,2016-12-31,466.0,False,False,False,False,False,True,False,True,False,True,False,False,False,False
230126,230126,2016-12-31,2907.0,False,False,False,False,False,True,False,True,False,False,True,False,False,False
230127,230127,2016-12-31,2299.0,False,False,False,False,False,True,False,True,False,False,False,True,False,False
230128,230128,2016-12-31,1242.0,False,False,False,False,False,True,False,True,False,False,False,False,True,False


We now have our data cleaned and ready. Let's import relevant sklearn libraries and use our SGD Resgressor

In [15]:
y_train = sticker_data_cleaned['num_sold']
y_train

1          973.0
2          906.0
3          423.0
4          491.0
5          300.0
           ...  
230125     466.0
230126    2907.0
230127    2299.0
230128    1242.0
230129    1622.0
Name: num_sold, Length: 221259, dtype: float64

In [16]:
X_train = sticker_data_cleaned.drop(columns=['id', 'num_sold', 'date'])
X_train

,Canada,Finland,Italy,Kenya,Norway,Singapore,Discount Stickers,Premium Sticker Mart,Stickers for Less,Holographic Goose,Kaggle,Kaggle Tiers,Kerneler,Kerneler Dark Mode
1,True,False,False,False,False,False,True,False,False,False,True,False,False,False
2,True,False,False,False,False,False,True,False,False,False,False,True,False,False
3,True,False,False,False,False,False,True,False,False,False,False,False,True,False
4,True,False,False,False,False,False,True,False,False,False,False,False,False,True
5,True,False,False,False,False,False,False,False,True,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230125,False,False,False,False,False,True,False,True,False,True,False,False,False,False
230126,False,False,False,False,False,True,False,True,False,False,True,False,False,False
230127,False,False,False,False,False,True,False,True,False,False,False,True,False,False
230128,False,False,False,False,False,True,False,True,False,False,False,False,True,False


In [17]:
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_absolute_percentage_error

# Using all defaults of the SGDRegressor class
sgd_regressor = SGDRegressor()
sgd_regressor.fit(X_train, y_train)

SGDRegressor()

Now that we have our regressor let's clean X_test to match X_train

In [18]:
X_test = pd.read_csv('/kaggle/input/forecasting-sticker-sales/test.csv')
X_test

,id,date,country,store,product
0,230130,2017-01-01,Canada,Discount Stickers,Holographic Goose
1,230131,2017-01-01,Canada,Discount Stickers,Kaggle
2,230132,2017-01-01,Canada,Discount Stickers,Kaggle Tiers
3,230133,2017-01-01,Canada,Discount Stickers,Kerneler
4,230134,2017-01-01,Canada,Discount Stickers,Kerneler Dark Mode
...,...,...,...,...,...
98545,328675,2019-12-31,Singapore,Premium Sticker Mart,Holographic Goose
98546,328676,2019-12-31,Singapore,Premium Sticker Mart,Kaggle
98547,328677,2019-12-31,Singapore,Premium Sticker Mart,Kaggle Tiers
98548,328678,2019-12-31,Singapore,Premium Sticker Mart,Kerneler


In [19]:
X_test = X_test.join(pd.get_dummies(X_test['country'])).drop(columns='country')
X_test

,id,date,store,product,Canada,Finland,Italy,Kenya,Norway,Singapore
0,230130,2017-01-01,Discount Stickers,Holographic Goose,True,False,False,False,False,False
1,230131,2017-01-01,Discount Stickers,Kaggle,True,False,False,False,False,False
2,230132,2017-01-01,Discount Stickers,Kaggle Tiers,True,False,False,False,False,False
3,230133,2017-01-01,Discount Stickers,Kerneler,True,False,False,False,False,False
4,230134,2017-01-01,Discount Stickers,Kerneler Dark Mode,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
98545,328675,2019-12-31,Premium Sticker Mart,Holographic Goose,False,False,False,False,False,True
98546,328676,2019-12-31,Premium Sticker Mart,Kaggle,False,False,False,False,False,True
98547,328677,2019-12-31,Premium Sticker Mart,Kaggle Tiers,False,False,False,False,False,True
98548,328678,2019-12-31,Premium Sticker Mart,Kerneler,False,False,False,False,False,True


In [20]:
X_test = X_test.join(pd.get_dummies(X_test['store'])).drop(columns='store')
X_test

,id,date,product,Canada,Finland,Italy,Kenya,Norway,Singapore,Discount Stickers,Premium Sticker Mart,Stickers for Less
0,230130,2017-01-01,Holographic Goose,True,False,False,False,False,False,True,False,False
1,230131,2017-01-01,Kaggle,True,False,False,False,False,False,True,False,False
2,230132,2017-01-01,Kaggle Tiers,True,False,False,False,False,False,True,False,False
3,230133,2017-01-01,Kerneler,True,False,False,False,False,False,True,False,False
4,230134,2017-01-01,Kerneler Dark Mode,True,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
98545,328675,2019-12-31,Holographic Goose,False,False,False,False,False,True,False,True,False
98546,328676,2019-12-31,Kaggle,False,False,False,False,False,True,False,True,False
98547,328677,2019-12-31,Kaggle Tiers,False,False,False,False,False,True,False,True,False
98548,328678,2019-12-31,Kerneler,False,False,False,False,False,True,False,True,False


In [21]:
X_test = X_test.join(pd.get_dummies(X_test['product'])).drop(columns='product')
X_test

,id,date,Canada,Finland,Italy,Kenya,Norway,Singapore,Discount Stickers,Premium Sticker Mart,Stickers for Less,Holographic Goose,Kaggle,Kaggle Tiers,Kerneler,Kerneler Dark Mode
0,230130,2017-01-01,True,False,False,False,False,False,True,False,False,True,False,False,False,False
1,230131,2017-01-01,True,False,False,False,False,False,True,False,False,False,True,False,False,False
2,230132,2017-01-01,True,False,False,False,False,False,True,False,False,False,False,True,False,False
3,230133,2017-01-01,True,False,False,False,False,False,True,False,False,False,False,False,True,False
4,230134,2017-01-01,True,False,False,False,False,False,True,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98545,328675,2019-12-31,False,False,False,False,False,True,False,True,False,True,False,False,False,False
98546,328676,2019-12-31,False,False,False,False,False,True,False,True,False,False,True,False,False,False
98547,328677,2019-12-31,False,False,False,False,False,True,False,True,False,False,False,True,False,False
98548,328678,2019-12-31,False,False,False,False,False,True,False,True,False,False,False,False,True,False


In [22]:
df = pd.to_datetime(X_test['date']).astype(int) // 10 ** 11
df

0        14832288
1        14832288
2        14832288
3        14832288
4        14832288
           ...   
98545    15777504
98546    15777504
98547    15777504
98548    15777504
98549    15777504
Name: date, Length: 98550, dtype: int64

In [23]:
X_test['date'] = pd.to_datetime(X_test['date']).astype(int) // 10 ** 11
X_test

,id,date,Canada,Finland,Italy,Kenya,Norway,Singapore,Discount Stickers,Premium Sticker Mart,Stickers for Less,Holographic Goose,Kaggle,Kaggle Tiers,Kerneler,Kerneler Dark Mode
0,230130,14832288,True,False,False,False,False,False,True,False,False,True,False,False,False,False
1,230131,14832288,True,False,False,False,False,False,True,False,False,False,True,False,False,False
2,230132,14832288,True,False,False,False,False,False,True,False,False,False,False,True,False,False
3,230133,14832288,True,False,False,False,False,False,True,False,False,False,False,False,True,False
4,230134,14832288,True,False,False,False,False,False,True,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98545,328675,15777504,False,False,False,False,False,True,False,True,False,True,False,False,False,False
98546,328676,15777504,False,False,False,False,False,True,False,True,False,False,True,False,False,False
98547,328677,15777504,False,False,False,False,False,True,False,True,False,False,False,True,False,False
98548,328678,15777504,False,False,False,False,False,True,False,True,False,False,False,False,True,False


In [24]:
X_test = X_test.drop(columns=['id', 'date'])
X_test

,Canada,Finland,Italy,Kenya,Norway,Singapore,Discount Stickers,Premium Sticker Mart,Stickers for Less,Holographic Goose,Kaggle,Kaggle Tiers,Kerneler,Kerneler Dark Mode
0,True,False,False,False,False,False,True,False,False,True,False,False,False,False
1,True,False,False,False,False,False,True,False,False,False,True,False,False,False
2,True,False,False,False,False,False,True,False,False,False,False,True,False,False
3,True,False,False,False,False,False,True,False,False,False,False,False,True,False
4,True,False,False,False,False,False,True,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98545,False,False,False,False,False,True,False,True,False,True,False,False,False,False
98546,False,False,False,False,False,True,False,True,False,False,True,False,False,False
98547,False,False,False,False,False,True,False,True,False,False,False,True,False,False
98548,False,False,False,False,False,True,False,True,False,False,False,False,True,False


We've cleaned X_test, now let's use our sgd_regressor

In [25]:
y_pred = sgd_regressor.predict(X_test)

Now that we have our predictions, let's create our csv to send submit.

In [26]:
X_test = pd.read_csv('/kaggle/input/forecasting-sticker-sales/test.csv')
X_test

,id,date,country,store,product
0,230130,2017-01-01,Canada,Discount Stickers,Holographic Goose
1,230131,2017-01-01,Canada,Discount Stickers,Kaggle
2,230132,2017-01-01,Canada,Discount Stickers,Kaggle Tiers
3,230133,2017-01-01,Canada,Discount Stickers,Kerneler
4,230134,2017-01-01,Canada,Discount Stickers,Kerneler Dark Mode
...,...,...,...,...,...
98545,328675,2019-12-31,Singapore,Premium Sticker Mart,Holographic Goose
98546,328676,2019-12-31,Singapore,Premium Sticker Mart,Kaggle
98547,328677,2019-12-31,Singapore,Premium Sticker Mart,Kaggle Tiers
98548,328678,2019-12-31,Singapore,Premium Sticker Mart,Kerneler


In [27]:
X_test = X_test['id']
X_test

0        230130
1        230131
2        230132
3        230133
4        230134
          ...  
98545    328675
98546    328676
98547    328677
98548    328678
98549    328679
Name: id, Length: 98550, dtype: int64

In [28]:
y_pred = pd.DataFrame(y_pred)
y_pred

,0
0,-237.183463
1,918.426847
2,711.017816
3,242.153822
4,343.795927
...,...
98545,485.746963
98546,1641.357273
98547,1433.948242
98548,965.084248


In [29]:
df = pd.DataFrame(X_test).join(y_pred)
df

,id,0
0,230130,-237.183463
1,230131,918.426847
2,230132,711.017816
3,230133,242.153822
4,230134,343.795927
...,...,...
98545,328675,485.746963
98546,328676,1641.357273
98547,328677,1433.948242
98548,328678,965.084248


In [30]:
df.rename(columns={0 : 'num_sold'}, inplace=True)
df

,id,num_sold
0,230130,-237.183463
1,230131,918.426847
2,230132,711.017816
3,230133,242.153822
4,230134,343.795927
...,...,...
98545,328675,485.746963
98546,328676,1641.357273
98547,328677,1433.948242
98548,328678,965.084248


In [31]:
df.to_csv('/kaggle/working/sticker_submission.csv', index=False)